In [2]:
import pandas as pd

In [3]:
log2fc_file = '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/3_IL2RAKO_Nascent/2_DE/1_compiled/nondupr_DEseq_log2fc.csv'
pval_file = '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/3_IL2RAKO_Nascent/2_DE/1_compiled/nondupr_DEseq_pval.csv'

log2fc_df = pd.read_csv(log2fc_file)
pval_df = pd.read_csv(pval_file)

In [4]:
log2fc_df

,transcript,gene_name,nondupr_KO_6h_vs_KO_0h,nondupr_KO_24h_vs_KO_0h,nondupr_KO_48h_vs_KO_0h,nondupr_WT_0h_vs_KO_0h,nondupr_WT_6h_vs_KO_6h,nondupr_WT_6h_vs_WT_0h,nondupr_WT_24h_vs_KO_24h,nondupr_WT_24h_vs_WT_0h,nondupr_WT_48h_vs_KO_48h,nondupr_WT_48h_vs_WT_0h
0,ENSMUSE00000869502,Gm1992,1.070701,-0.467832,-1.139993,0.005269,0.356765,1.422196,-1.637532,-2.110633,-0.324971,-1.470232
1,ENSMUSE00001341225,Gm7357,-1.549259,-3.184898,-1.397429,0.253213,0.165613,-1.636859,0.712847,-2.725263,-0.695018,-2.345659
2,ENSMUSE00001345397,Gm6123,-0.052769,-0.893124,-0.966981,-0.025068,-0.307340,-0.335041,-0.058811,-0.926868,0.334064,-0.607850
3,ENSMUSE00000837314,Lypla1,0.123428,0.032016,0.225477,0.009051,0.010183,0.124560,-0.005286,0.017679,0.016631,0.233057
4,ENSMUSE00001255376,Lypla1,0.107082,0.015221,0.210850,0.002615,0.013033,0.117500,-0.010226,0.002380,0.013067,0.221301
...,...,...,...,...,...,...,...,...,...,...,...,...
68922,ENSMUSE00000645388,Gm20775,-0.141755,0.537157,0.227121,0.343598,0.686185,0.200833,-0.115156,0.078404,-0.027572,-0.144048
68923,ENSMUSE00001342422,Gm37158,0.319709,-1.544699,-0.928623,0.551535,0.648958,0.417131,1.541793,-0.554441,0.144295,-1.335864
68924,ENSMUSE00001328317,Gm29644,1.403250,-0.710540,-0.942039,-3.401025,-4.486063,0.000000,-1.934630,0.000000,-2.679503,0.000000
68925,ENSMUSE00001322936,Gm28930,1.227445,2.261920,1.080878,-1.310619,-0.475315,2.062750,-0.475486,3.097053,0.357897,2.749394
